In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Setup OpenAI API key
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from typing import List
from pydantic import BaseModel, Field

### Pydantic syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic.

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [4]:
class User:
    def __init__(self, name: str, age: int, email: str) -> None:
        self.name = name
        self.age = age
        self.email = email

In [5]:
class PUser(BaseModel):
    name: str
    age: int
    email: str

In [ ]:
# This is expected to fail
foo_p = PUser(name="Jane", age="bar", email="jane@gmail.com")

### Pydantic to OpenAI function calling

In [8]:
class WeatherSearch(BaseModel):
    """Call this with location to get the weather at a place."""    
    location: str = Field(description="Location for which to get the weather for")

In [11]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

weather_function = convert_pydantic_to_openai_function(WeatherSearch, name="weather_search")

In [12]:
weather_function

{'name': 'weather_search',
 'description': 'Call this with location to get the weather at a place.',
 'parameters': {'description': 'Call this with location to get the weather at a place.',
  'properties': {'location': {'description': 'Location for which to get the weather for',
    'title': 'Location',
    'type': 'string'}},
  'required': ['location'],
  'title': 'WeatherSearch',
  'type': 'object'}}

In [13]:
from langchain.chat_models import ChatOpenAI

In [14]:
model = ChatOpenAI(temperature=0, verbose=True)

In [16]:
model.invoke("What is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "location": "San Francisco"\n}', 'name': 'weather_search'}})

In [18]:
model_with_function = model.bind(functions=[weather_function])
model_with_function.invoke("What is the weather in Denver today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "location": "Denver"\n}', 'name': 'weather_search'}})

### Using in a chain


In [19]:
from langchain.prompts import ChatPromptTemplate

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}")
    ]
)

In [21]:
chain = prompt | model_with_function

In [22]:
chain.invoke({"input": "What is the weather in NY today?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "location": "New York"\n}', 'name': 'weather_search'}})